<h1 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
    DeepSeek-R1 with Amazon Bedrock 
</h1>

In [1]:
# !pip install huggingface_hub boto3

# Download the model weights (`DeepSeek-R1-Distill-Qwen-14B`)

### Step 1: Download the model weights locally from Hugging Face

In [2]:
from huggingface_hub import snapshot_download

# Model ID
repo_id = 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B'

# Downloading the model weights locally from Hugging Face
local_dir = snapshot_download(repo_id)
print(f"Model downloaded to: {local_dir}")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Model downloaded to: /home/sagemaker-user/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/24ae87a9c340aa4207dd46509414c019998e0161


### Step 2: Upload the weights on S3

In [5]:
import boto3
import os

# Uploading the model files to the specified S3 bucket
bucket_name = 'llm-weights-demo'
target_folder = 'models/DeepSeek-R1-Distill-Llama-8B'

s3 = boto3.client('s3')

for root, _, files in os.walk(local_dir):
    for file in files:
        file_path = os.path.join(root, file)
        relative_path = os.path.relpath(file_path, local_dir)
        s3_key = f"{target_folder}/{relative_path}"  
        print(f"Uploading {file_path} to s3://{bucket_name}/{s3_key}")
        s3.upload_file(file_path, bucket_name, s3_key)

print("Model uploaded to S3 successfully.")

Uploading /home/sagemaker-user/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/24ae87a9c340aa4207dd46509414c019998e0161/config.json to s3://llm-weights-demo/models/DeepSeek-R1-Distill-Llama-8B/config.json
Uploading /home/sagemaker-user/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/24ae87a9c340aa4207dd46509414c019998e0161/README.md to s3://llm-weights-demo/models/DeepSeek-R1-Distill-Llama-8B/README.md
Uploading /home/sagemaker-user/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/24ae87a9c340aa4207dd46509414c019998e0161/LICENSE to s3://llm-weights-demo/models/DeepSeek-R1-Distill-Llama-8B/LICENSE
Uploading /home/sagemaker-user/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/24ae87a9c340aa4207dd46509414c019998e0161/generation_config.json to s3://llm-weights-demo/models/DeepSeek-R1-Distill-Llama-8B/generation_config.json
Uploading /home/sagemaker-user

# Import the model on Amazon Bedrock 

- Go to the `AWS console` and, on the left-hand size, click on **Imported models under Foundation models**.
- Click on **Import model**
- Use `my-DeepSeek-R1-Distill-Llama-8B` as the Model name, and enter the S3 location from above. Click on Import model 


![img1](imgs/img1.png)

- Wait for the model to get import **sucessfully**

  ![img2](imgs/img2.png)

- Take a note of the **Model ARN**

  ![img3](imgs/img3.png)



# Test the model 

In [24]:
from botocore.config import Config
import json

def invoke_bedrock_model(model_arn, message, region_name='us-east-1'):

    model_id = model_arn

    config = Config(
        retries={
            'total_max_attempts': 10, 
            'mode': 'standard'
        }
    )

    session = boto3.session.Session()
    br_runtime = session.client(service_name = 'bedrock-runtime', 
                                     region_name=region_name, 
                                     config=config)
        
    try:
        invoke_response = br_runtime.invoke_model(modelId=model_id, 
                                                body=json.dumps({'prompt': message}), 
                                                accept="application/json", 
                                                contentType="application/json")
        result = invoke_response["body"] = json.loads(invoke_response["body"].read().decode("utf-8"))
    except Exception as e:
        print(e)
        print(e.__repr__())

    return result



In [29]:
model_arn = 'arn:aws:bedrock:us-east-1:507922848584:imported-model/0j54azgqtt5l'

prompt = "A bat and a ball together cost $10.50. The bat costs $2.25 more than the ball. How much does the ball cost?"

response = invoke_bedrock_model(model_arn=model_arn, message=prompt)


In [30]:
from IPython import display

display.Markdown(response['generation'])

 

First, let me define the variables. Let’s say the cost of the ball is \( b \). Then, the cost of the bat would be \( b + 2.25 \).

Since the total cost is $10.50, I can set up the equation: \( b + (b + 2.25) = 10.50 \).

Combining like terms, this becomes \( 2b + 2.25 = 10.50 \).

Subtracting 2.25 from both sides gives \( 2b = 8.25 \).

Dividing both sides by 2, I find that \( b = 4.125 \).

So, the ball costs $4.125.
</think>

Let's solve the problem step by step.

**Given:**
- The total cost of a bat and a ball together is \$10.50.
- The bat costs \$2.25 more than the ball.

**Let:**
- \( b \) = Cost of the ball (\$)
- \( b + 2.25 \) = Cost of the bat (\$)

**Set up the equation:**
\[
b + (b + 2.25) = 10.50
\]

**Combine like terms:**
\[
2b + 2.25 = 10.50
\]

**Subtract 2.25 from both sides:**
\[
2b = 10.50 - 2.25
\]
\[
2b = 8.25
\]

**Divide both sides by 2:**
\[
b = \frac{8.25}{2}
\]
\[
b = 4.125
\]

**Answer:**
The ball costs \(\boxed{4.125}\) dollars.